In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pathlib import Path

# URL HTML com as duas abas publicadas
URL_HTML = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSa06AfYC_ufn2vqTsYZkeyX7QvNwYH2gdH8VPcZOCS8_pyOJzTOtU3tufy_OL5sWhVvW_qm3mBpWJr/pubhtml"

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

def parse_num_pt(x):
    """Converte '1.234,56' -> 1234.56 mantendo NaN quando vazio."""
    if pd.isna(x): 
        return np.nan
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if not s:
        return np.nan
    s = s.replace('.', '').replace(',', '.')
    try:
        return float(s)
    except Exception:
        return np.nan

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    """Remove colunas 'Unnamed', linhas-cabeçalho duplicadas e normaliza nomes."""
    df = df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed")]
    # remove linhas que repetem o cabeçalho
    if len(df):
        mask = df.astype(str).apply(lambda s: (list(s.values) == list(df.columns.astype(str))), axis=1)
        if mask.any():
            df = df.loc[~mask]
    df.columns = [re.sub(r'\s+', ' ', c).strip() for c in df.columns.astype(str)]
    return df.reset_index(drop=True)


In [2]:
# Lê todas as tabelas presentes na página HTML.
# Usamos decimal=',' e thousands='.' para ajudar a interpretar números europeus.
tables = pd.read_html(URL_HTML, flavor="bs4", displayed_only=True, decimal=',', thousands='.')

print(f"Foram lidas {len(tables)} tabelas do HTML.\n")
for i, t in enumerate(tables):
    print(f"[{i}] shape={t.shape} | colunas={list(t.columns)}")

# Assinaturas esperadas (podem variar levemente; ajuste se mudar seu cabeçalho)
cols_resumo_expect = {"Semana", "Canal", "Pães", "Receita (€)"}
cols_pedidos_expect = {
    "Data","Semana","Cliente","Região","Quantidade (pães)",
    "Tipo (Fresco/MAP)","Canal","Valor Unitário (€)","Valor Total (€)","Status (Pago/Deve)"
}

def score(df_cols, expected):
    cset = set(map(str, df_cols))
    return len(cset & expected), len(expected - cset)

idx_resumo = idx_pedidos = None
best_r = (-1, 999)
best_p = (-1, 999)

for i, t in enumerate(tables):
    sr = score(t.columns, cols_resumo_expect)
    sp = score(t.columns, cols_pedidos_expect)
    if sr > best_r and sr[0] >= 3:
        best_r = sr; idx_resumo = i
    if sp > best_p and sp[0] >= 6:
        best_p = sp; idx_pedidos = i

print("\nProvável índice do RESUMO:", idx_resumo, " | score:", best_r)
print("Provável índice dos PEDIDOS:", idx_pedidos, " | score:", best_p)

if idx_resumo is None or idx_pedidos is None:
    raise ValueError("Não foi possível identificar as tabelas de Resumo/Pedidos no HTML. Revise cabeçalhos/assinaturas.")


ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

In [ ]:
df_resumo = clean_df(tables[idx_resumo].copy())
df_pedidos = clean_df(tables[idx_pedidos].copy())

# Conversões numéricas
if 'Receita (€)' in df_resumo.columns:
    df_resumo['Receita (€)'] = df_resumo['Receita (€)'].apply(parse_num_pt)
if 'Pães' in df_resumo.columns:
    df_resumo['Pães'] = pd.to_numeric(df_resumo['Pães'], errors='coerce').astype('Int64')

for col in ['Valor Unitário (€)', 'Valor Total (€)']:
    if col in df_pedidos.columns:
        df_pedidos[col] = df_pedidos[col].apply(parse_num_pt)
if 'Quantidade (pães)' in df_pedidos.columns:
    df_pedidos['Quantidade (pães)'] = pd.to_numeric(df_pedidos['Quantidade (pães)'], errors='coerce').astype('Int64')

# Normaliza tipo e flag ATM
tipo_col = 'Tipo (Fresco/MAP)'
if tipo_col in df_pedidos.columns:
    df_pedidos[tipo_col] = df_pedidos[tipo_col].astype(str).str.strip()
else:
    df_pedidos[tipo_col] = 'Fresco'
df_pedidos['is_atm'] = df_pedidos[tipo_col].str.upper().eq('ATM')

# Totais rápidos
total_paes = int(df_pedidos['Quantidade (pães)'].sum())
total_receita = float(df_pedidos['Valor Total (€)'].sum())
print("=== Totais (Pedidos) ===")
print(f"Pães: {total_paes}")
print(f"Receita (€): {round(total_receita,2)}")

display(df_resumo.head())
display(df_pedidos.head())


In [ ]:
# ---------------- Premissas (Word) ----------------
# Ingredientes + embalagem (saco+etiqueta) por pão
CUSTO_INGREDIENTES = 0.05       # €/pão
EMBALAGEM_SACO_10  = 0.17       # €/saco com 10 pães
ETIQUETA_SACO_10   = 0.02       # €/saco com 10 pães
EMBALAGEM_POR_PAO  = (EMBALAGEM_SACO_10 + ETIQUETA_SACO_10)/10.0
COGS_BASE          = CUSTO_INGREDIENTES + EMBALAGEM_POR_PAO   # €/pão

# Logística / comissão
COMISSAO_ANINHA  = 0.20         # €/pão (D2C entrega)
NACEX_ENVIO_10KG = 5.50         # € por envio (10kg tier)
CUSTO_CAIXA      = 0.60         # € por caixa econômica
PAES_POR_CAIXA   = 70           # ~7 sacos de 10 pães
NACEX_POR_PAO    = (NACEX_ENVIO_10KG + CUSTO_CAIXA)/PAES_POR_CAIXA  # €/pão

# MAP (ATM) — custo por pão em dois cenários
MAP_COST_LOW  = 0.012           # €/pão (alto volume)
MAP_COST_HIGH = 0.050           # €/pão (baixo volume)

# Custos fixos mensais (aluguel, utilidades, salários)
CUSTOS_FIXOS_MES = 2700.0       # €

# Fallback de preços por canal (usamos a mediana observada se existir)
FALLBACK_PRECOS = {
    'Aninha': 0.70,
    'Recolha': 0.50,
    'Boa Turma': 0.40,
    'NACEX': 0.45,
}


In [ ]:
def obter_precos_por_canal(df):
    """Mediana do preço por canal vinda do próprio dataset; se faltar, usa fallback."""
    precos = df.groupby('Canal')['Valor Unitário (€)'].median().to_dict()
    for k,v in FALLBACK_PRECOS.items():
        precos.setdefault(k, v)
    return precos

def cogs_por_pao(canal, is_atm: bool, scenario='low'):
    """COGS = base + adders (comissão/entrega) + MAP (se ATM)."""
    add = 0.0
    if canal == 'Aninha':
        add += COMISSAO_ANINHA
    if canal == 'NACEX':
        add += NACEX_POR_PAO
    map_cost = 0.0
    if is_atm:
        map_cost = MAP_COST_LOW if scenario=='low' else MAP_COST_HIGH
    return COGS_BASE + add + map_cost

def unit_economics_tabela(df, precos_por_canal):
    rows = []
    canais = sorted(df['Canal'].dropna().unique())
    for ch in canais:
        preco = precos_por_canal.get(ch, np.nan)
        for is_atm in [False, True]:
            label = f"{ch} - {'ATM' if is_atm else 'Fresco'}"
            cogs_l = cogs_por_pao(ch, is_atm, 'low')
            cogs_h = cogs_por_pao(ch, is_atm, 'high')
            marg_l = preco - cogs_l if pd.notna(preco) else np.nan
            marg_h = preco - cogs_h if pd.notna(preco) else np.nan
            rows.append({
                'Canal/Tipo': label,
                'Preço (€)': round(preco,3) if pd.notna(preco) else np.nan,
                'COGS baixo (€)': round(cogs_l,3),
                'COGS alto (€)': round(cogs_h,3),
                'Margem baixo (€)': round(marg_l,3) if pd.notna(marg_l) else np.nan,
                'Margem alto (€)': round(marg_h,3) if pd.notna(marg_h) else np.nan,
                'Margem baixo (%)': round(100*marg_l/preco,1) if preco else np.nan,
                'Margem alto (%)': round(100*marg_h/preco,1) if preco else np.nan,
            })
    return pd.DataFrame(rows)

def consolidar_contribuicao(df):
    """Calcula contribuição por canal em dois cenários (MAP baixo/alto) e aloca fixos por volume."""
    df = df.copy()
    for sc in ['low','high']:
        df[f'COGS/pão {sc}'] = df.apply(lambda r: cogs_por_pao(r['Canal'], bool(r['is_atm']), sc), axis=1)
        df[f'Margem/pão {sc}'] = df['Valor Unitário (€)'] - df[f'COGS/pão {sc}']
        df[f'Contribuição {sc}'] = df[f'Margem/pão {sc}'] * df['Quantidade (pães)']
    contri = (df.groupby('Canal', as_index=False)
                .agg(paes=('Quantidade (pães)','sum'),
                     receita=('Valor Total (€)','sum'),
                     contrib_baixa=('Contribuição low','sum'),
                     contrib_alta=('Contribuição high','sum')))
    total_p = contri['paes'].sum()
    contri['FixedCost alloc (€)'] = CUSTOS_FIXOS_MES * (contri['paes']/total_p) if total_p else 0.0
    contri['Lucro op (baixo) €'] = contri['contrib_baixa'] - contri['FixedCost alloc (€)']
    contri['Lucro op (alto)  €'] = contri['contrib_alta']  - contri['FixedCost alloc (€)']
    return contri

def calcular_break_even(contri_df):
    """CM e Break-even mensal:
       CM = Contribuição total / Pães totais
       BE (pães/mês) = Custos Fixos / CM
    """
    total_contrib_low  = contri_df['contrib_baixa'].sum()
    total_contrib_high = contri_df['contrib_alta'].sum()
    total_paes         = contri_df['paes'].sum()
    cm_low  = total_contrib_low / total_paes if total_paes else np.nan
    cm_high = total_contrib_high / total_paes if total_paes else np.nan
    be_low  = CUSTOS_FIXOS_MES / cm_low if cm_low else np.nan
    be_high = CUSTOS_FIXOS_MES / cm_high if cm_high else np.nan
    return cm_low, cm_high, be_low, be_high


In [ ]:
precos_canais = obter_precos_por_canal(df_pedidos)

unit_df = unit_economics_tabela(df_pedidos, precos_canais)
display(unit_df)

contri_df = consolidar_contribuicao(df_pedidos)
display(contri_df.round(2))


In [ ]:
cm_low, cm_high, be_low, be_high = calcular_break_even(contri_df)

print("=== CM e Break-even (mês) ===")
print(f"CM €/pão (MAP baixo): {None if pd.isna(cm_low) else round(cm_low,3)}")
print(f"CM €/pão (MAP alto):  {None if pd.isna(cm_high) else round(cm_high,3)}")
print(f"Break-even pães/mês (MAP baixo): {None if pd.isna(be_low) else int(round(be_low,0))}")
print(f"Break-even pães/mês (MAP alto):  {None if pd.isna(be_high) else int(round(be_high,0))}")

# Gráfico — mix por canal (pães)
mix = (df_pedidos.groupby('Canal', as_index=False)
       .agg(paes=('Quantidade (pães)','sum'),
            receita=('Valor Total (€)','sum'))
       .sort_values('paes', ascending=False))

plt.figure(figsize=(7,4))
plt.bar(mix['Canal'], mix['paes'])
plt.title('Mix de Pães por Canal')
plt.xlabel('Canal'); plt.ylabel('Pães')
plt.xticks(rotation=25)
plt.tight_layout()
plt.show()

# Gráfico — evolução semanal (pães)
sem = (df_pedidos.groupby('Semana', as_index=False)
       .agg(paes=('Quantidade (pães)','sum'),
            receita=('Valor Total (€)','sum'))
       .sort_values('Semana'))

plt.figure(figsize=(7,4))
plt.plot(sem['Semana'], sem['paes'], marker='o')
plt.title('Evolução Semanal de Pães')
plt.xlabel('Semana'); plt.ylabel('Pães')
plt.tight_layout()
plt.show()


In [ ]:
unit_path   = OUT_DIR / "unit_economics_canais.csv"
contri_path = OUT_DIR / "contribuicao_canais.csv"
mix_path    = OUT_DIR / "mix_canal.csv"
sem_path    = OUT_DIR / "evolucao_semanal.csv"

unit_df.to_csv(unit_path, index=False, encoding="utf-8-sig")
contri_df.round(2).to_csv(contri_path, index=False, encoding="utf-8-sig")
mix.round(2).to_csv(mix_path, index=False, encoding="utf-8-sig")
sem.round(2).to_csv(sem_path, index=False, encoding="utf-8-sig")

print("Arquivos exportados em:", OUT_DIR.resolve())
for p in [unit_path, contri_path, mix_path, sem_path]:
    print("-", p.name)
